In [1]:
import pandas as pd
df = pd.read_csv('../files/Stuff.csv', encoding='utf-8')
product_id = pd.read_csv('../files/product_id.csv', encoding='utf-8')
num_to_words = {
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    0: '0',
    '-': '-',
    '+': '+',
    '.': '__',
}

In [2]:
def extract_data(num):
    code = num // 10000000000
    rest = num % 10000000000

    sph_sign = '-' if rest // 1000000000 == 0 else '+'
    rest = rest % 1000000000

    sph = (rest // 100000)/100
    rest = rest % 100000

    cyl_sign = '-' if rest // 10000 == 0 else '+'
    rest = rest % 10000

    cyl = rest/100

    return code, sph_sign,sph,cyl_sign,cyl

# Apply the function to 'StuffCode' and expand the result into two columns
df[['code', 'sph_sign','sph','cyl_sign','cyl']] = pd.DataFrame(df['StuffCode'].apply(extract_data).tolist(), index=df.index)

In [3]:
# Convert sph_sign, sph, cyl_sign, cyl using num_to_words dictionary
def convert_to_words(prefix, sign, value):
    # Convert both sign and value to their respective string representations
    sign_str = num_to_words.get(sign, sign)
    value_str = ''.join([num_to_words.get(char, char) for char in str(abs(value)) if char in '0123456789.'])
    # Prepend the prefix to the result
    return f"{prefix}_{sign_str}_{value_str}"

# Apply the function to create 'sphere' and 'cylinder' columns
df['sphere'] = df.apply(lambda x: convert_to_words("sph", x['sph_sign'], x['sph']), axis=1)
df['cylinder'] = df.apply(lambda x: convert_to_words("cyl", x['cyl_sign'], x['cyl']), axis=1)

df.columns = df.columns.str.strip()
product_id.columns = product_id.columns.str.strip()
merged_df = pd.merge(df, product_id[['product_name', 'id']], left_on='StuffMainGroupName', right_on='product_name', how='left')
merged_df.to_csv('merged_df.csv', index=False,encoding='utf-8')

In [4]:
# Strip whitespace from 'sphere' and 'cylinder' columns
merged_df['sphere'] = merged_df['sphere'].astype(str).str.strip()
merged_df['cylinder'] = merged_df['cylinder'].astype(str).str.strip()

# Drop specified columns
merged_df.to_csv('merged_df_complete.csv', index=False, encoding='utf-8')
merged_df = merged_df.drop(['StuffModel', 'code', 'sph_sign', 'sph', 'cyl_sign', 'cyl'], axis=1)


# Remove duplicates
merged_df = merged_df.drop_duplicates()

# Save to CSV
merged_df.to_csv('merged_df.csv', index=False, encoding='utf-8')
